In [8]:
import pandas as pd
import altair as alt

In [15]:
df = pd.read_csv("../data/corin_data.csv")
df['dataTime'] = pd.to_datetime(df['dataTime'], format='%d.%m.%Y %H:%M:%S')
df.tail()

,dataTime,total_confirmed,total_death,total_recovered,total_hosp,total_intens,total_tests,B_confirmed,B_death,B_recovered,...,V_hosp,V_intens,W_confirmed,W_death,W_recovered,W_hosp,W_intens,remarks,source,viewedTime
94,2020-04-12 09:30:00,13945,350,6987.0,1036.0,246.0,140975,260.0,6.0,120.0,...,38.0,11.0,2029.0,74.0,533.0,187.0,37.0,corin.at bmsgpk_loader v0.3,https://www.sozialministerium.at/Informationen...,12.04.2020 11:13:35
95,2020-04-12 09:30:00,13945,350,6987.0,1035.0,243.0,144877,260.0,6.0,120.0,...,37.0,10.0,2029.0,74.0,533.0,191.0,35.0,corin.at bmsgpk_loader v0.3,https://www.sozialministerium.at/Informationen...,12.04.2020 17:41:34
96,2020-04-12 15:00:00,13894,350,6987.0,1035.0,243.0,144877,260.0,6.0,120.0,...,37.0,10.0,2000.0,74.0,533.0,191.0,35.0,corin.at bmsgpk_loader v0.3,https://www.sozialministerium.at/Informationen...,13.04.2020 07:20:25
97,2020-04-13 08:00:00,13942,368,7343.0,981.0,239.0,148412,266.0,6.0,134.0,...,32.0,10.0,2011.0,75.0,551.0,163.0,34.0,corin.at bmsgpk_loader v0.3,https://www.sozialministerium.at/Informationen...,13.04.2020 11:06:23
98,2020-04-13 15:00:00,13999,368,7343.0,981.0,239.0,148412,268.0,6.0,134.0,...,32.0,10.0,2015.0,75.0,551.0,163.0,34.0,corin.at bmsgpk_loader v0.3,https://www.sozialministerium.at/Informationen...,13.04.2020 16:30:02


In [16]:
counties = ['B', 'K', 'Noe', 'Ooe', 'S', 'St', 'T', 'V', 'W']

In [ ]:
df_by_day = df

In [125]:
def c_line(att='total_confirmed',col='blue',scaleType='linear'):
    return alt.Chart(df, width=300, height=200).transform_filter(
        alt.datum[att] > 0  
    ).transform_timeunit(
        date='yearmonthdate(dataTime)'
    ).transform_aggregate(
        max_att='max('+att+')',
        groupby=['date']
    ).mark_line(
        color=col,
        size=2
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('max_att:Q', scale=alt.Scale(type=scaleType, base=10), title=''),
        tooltip=['date:T', 'max_att:Q']
    )

confirmed_cases = c_line('total_confirmed', '#1f77b4')
deaths = c_line('total_death', '#d62728')
recovered = c_line('total_recovered', '#ff7f0e')

confirmed_cases_log = c_line('total_confirmed', '#1f77b4', 'log')
deaths_log = c_line('total_death', '#d62728', 'log')
recovered_log = c_line('total_recovered', '#ff7f0e', 'log')

linear_scale = alt.layer(confirmed_cases, deaths, recovered).properties(title='Linear Scale')
log_scale = alt.layer(confirmed_cases_log, deaths_log, recovered_log).properties(title='Log Scale')

(linear_scale | log_scale).resolve_scale(color='independent')

alt.HConcatChart(...)

In [155]:
day = alt.Chart(df, width=300, height=200
).mark_line(
    color='#1f77b4',
    size=2
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_confirmed='max(total_confirmed)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_confirmed)',
    rolling_min='min(max_total_confirmed)',
    frame=[-1, 0]
).transform_calculate(
    diff='datum.rolling_max - datum.rolling_min'
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', title=''),
    tooltip=['date:T', 'diff:Q']
)

ma = alt.Chart(df, width=300, height=200
).mark_line(
    color='lightblue',
    size=2
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_confirmed='max(total_confirmed)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_confirmed)',
    rolling_min='min(max_total_confirmed)',
    frame=[-1, 0]
).transform_calculate(
    diff='datum.rolling_max - datum.rolling_min'
).transform_window(
    rolling_mean='mean(diff)',
    frame=[-6, 0]
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', title=''),
    tooltip=['date:T', 'rolling_mean:Q']
)

new_cases_by_day = (ma + day).properties(title='Neue Fälle pro Tag')

day_percent = alt.Chart(df, width=300, height=200
).mark_line(
    color='#1f77b4',
    size=2
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_confirmed='max(total_confirmed)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_confirmed)',
    rolling_min='min(max_total_confirmed)',
    frame=[-1, 0]
).transform_calculate(
    diff='(datum.rolling_max - datum.rolling_min) / datum.rolling_max * 100'
).encode(
    alt.X('date:T', title=''),
    alt.Y('diff:Q', title=''),
    tooltip=['date:T', 'diff:Q']
)

ma_percent = alt.Chart(df, width=300, height=200
).mark_line(
    color='lightblue',
    size=2
).transform_timeunit(
    date='yearmonthdate(dataTime)'
).transform_aggregate(
    max_total_confirmed='max(total_confirmed)',
    groupby=['date']
).transform_window(
    rolling_max='max(max_total_confirmed)',
    rolling_min='min(max_total_confirmed)',
    frame=[-1, 0]
).transform_calculate(
    diff='(datum.rolling_max - datum.rolling_min) / datum.rolling_max * 100'
).transform_window(
    rolling_mean='mean(diff)',
    frame=[-6, 0]
).encode(
    alt.X('date:T', title=''),
    alt.Y('rolling_mean:Q', title=''),
    tooltip=['date:T', 'rolling_mean:Q']
)

new_cases_percent_by_day = (ma_percent + day_percent).properties(title='Neue Fälle pro Tag Steigerung %')

new_cases_by_day | new_cases_percent_by_day

alt.HConcatChart(...)